In [35]:
import numpy as np
import pandas as pd

from xopto.mcml import mc
from xopto.cl import clinfo
from xopto.materials import skin
from tqdm import trange

In [36]:
WL = np.arange(400, 801, 10)                     # 41 points, 400-800 nm
N_PER_CLASS = 1500
N_PHOTONS = 1e6

In [37]:
RNG = np.random.default_rng()

CUBE_NORMAL = [
    dict(melanin=(0.001, 0.08),  water=(0.015, 0.025)),                   # layer 1
    dict(water  =(0.55 , 0.75),  blood=(0.01 , 0.06), spo2 =(0.95, 0.99)),# layer 2
    dict(water  =(0.03 , 0.07),  fat  =(0.25, 0.55), blood=(0.01, 0.05),
            spo2   =(0.95 , 0.99))]                                          # layer 3

CUBE_HEMANGIOMA = [
    dict(melanin=(0.001, 0.08),  water=(0.015, 0.025)),
    dict(water  =(0.55 , 0.75),  blood=(0.12 , 0.25), spo2 =(0.75, 0.85)),
    dict(water  =(0.03 , 0.07),  fat  =(0.25, 0.55), blood=(0.01, 0.05),
            spo2   =(0.95 , 0.99))]

In [38]:
def random_layer(layer_dict):
    return {k: RNG.uniform(low, high) for k, (low, high) in layer_dict.items()}

In [39]:
cl_device = clinfo.gpu()

# DEFINE SOURCE
source = mc.mcsource.Line(
    position=(0.0, 0.0, 0.0),
    direction=(0.0, 0.0, 1.0)
)

# DEFINE A DETECTOR FOR INTEGRATING SPHERE
sp_r = 0.5e-2  # integrating sphere opening in m
detector_top = mc.mcdetector.Radial(
    mc.mcdetector.RadialAxis(
        start=0.0,
        stop=2*sp_r,
        n=2)
)
detectors = mc.mcdetector.Detectors(
    top=detector_top
)

In [40]:
def make_sample(cube):
    mdl = skin.Skin3()                          # three-layer object
    # --- epidermis -----------------------------------------------------
    l1 = random_layer(cube[0])
    mdl[0].melanin = l1['melanin']
    mdl[0].water   = l1['water']
    # --- dermis --------------------------------------------------------
    l2 = random_layer(cube[1])
    mdl[1].water = l2['water']
    mdl[1].blood = l2['blood']
    mdl[1].spo2  = l2['spo2']
    # --- sub-cutis -----------------------------------------------------
    l3 = random_layer(cube[2])
    mdl[2].water = l3['water']
    mdl[2].fat   = l3['fat']
    mdl[2].blood = l3['blood']
    mdl[2].spo2  = l3['spo2']

    reflectance_spectrum = np.zeros(len(WL), dtype=np.float64)
    for i, wl in enumerate(WL):
        wl_m = wl * 1e-9  # convert nm to m
        layers = mdl.create_mc_layers(wl_m)
        mc_obj = mc.Mc(layers, source, detectors, cl_devices=cl_device)
        mc_obj.rmax = 50.0e-3

        detector = mc_obj.run(N_PHOTONS, verbose=False)[-1]
        reflectance_spectrum[i] = detector.top.reflectance[0] * np.pi * sp_r**2

    return reflectance_spectrum

In [41]:
spectra  = []
labels   = []

for _ in trange(N_PER_CLASS, desc="Simulating"):
    spectra.append(make_sample(CUBE_NORMAL));     
    labels.append(0)
    spectra.append(make_sample(CUBE_HEMANGIOMA)); 
    labels.append(1)

df = pd.DataFrame(np.vstack(spectra),
                    columns=[f"R_{l}nm" for l in WL])
df['hemangioma'] = labels

df.to_csv("skin3_synthetic_reflectance_2.csv", index=False)
print("Saved skin3_synthetic_reflectance_2.csv  (shape:", df.shape, ")")

Simulating:   0%|          | 0/1500 [00:00<?, ?it/s]

/home/mikhail/Projects/Skoltech/Biophotonics/light-in-skin-animation/pyxopto/xopto/materials/absorption/base.py:137: UserWarning: Wavelength is out of valid range [100.0, 799.5] nm!
  warnings.warn(
Simulating: 100%|██████████| 1500/1500 [6:13:37<00:00, 14.95s/it]    

Saved skin3_synthetic_reflectance_2.csv  (shape: (3000, 42) )
